# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_city_df = pd.read_csv("../output_data/cities.csv")
#vac_city_df.head()
del vac_city_df['City_ID']
vac_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,severo-kurilsk,50.68,156.12,37.22,77,100,14.54,RU,1604881196
1,hermanus,-34.42,19.23,51.01,90,43,11.12,ZA,1604881196
2,muros,42.78,-9.06,55.40,93,90,12.75,ES,1604881196
3,mahebourg,-20.41,57.70,73.40,88,40,14.56,MU,1604881196
4,puerto ayora,-0.74,-90.35,69.01,84,11,7.00,EC,1604881196


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vac_city_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = vac_city_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_df = vac_city_df.loc[(vac_city_df["Wind Speed"] <= 10) & (vac_city_df["Cloudiness"] <=10) & \
                                   (vac_city_df["Max Temp"] >= 70) & (vac_city_df["Max Temp"] <= 80)
                                & (vac_city_df["Humidity"] >= 10) & (vac_city_df["Humidity"] <= 85)].dropna()

narrow_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,hobart,-42.88,147.33,71.01,48,5,5.82,AU,1604881063
41,aojiang,22.94,116.04,73.00,48,4,1.99,CN,1604881049
75,sao filipe,14.90,-24.50,79.79,74,7,9.95,CV,1604881206
97,saint-philippe,-21.36,55.77,73.40,73,0,4.70,RE,1604881209
121,ambovombe,-25.17,46.08,70.14,82,0,5.95,MG,1604881212
215,sitrah,26.15,50.62,78.80,83,5,4.70,BH,1604881223
228,irece,-11.30,-41.86,74.10,67,0,9.73,BR,1604881225
243,hambantota,6.12,81.12,78.84,81,0,5.14,LK,1604881226
257,new norfolk,-42.78,147.06,71.01,43,4,1.01,AU,1604881228
282,wonthaggi,-38.60,145.59,78.01,58,0,1.99,AU,1604881231


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
11,hobart,AU,-42.88,147.33,
41,aojiang,CN,22.94,116.04,
75,sao filipe,CV,14.90,-24.50,
97,saint-philippe,RE,-21.36,55.77,
121,ambovombe,MG,-25.17,46.08,
215,sitrah,BH,26.15,50.62,
228,irece,BR,-11.30,-41.86,
243,hambantota,LK,6.12,81.12,
257,new norfolk,AU,-42.78,147.06,
282,wonthaggi,AU,-38.60,145.59,


In [7]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {'key': g_key,
          'type': 'lodging',
          'radius': 5000
             }


for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    city_name = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field... skipping.")
        
    print("-------------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

print("-----End of Search-----")

Retrieving Results for Index 11: hobart.
Closest hotel in hobart is St Ives Hobart Accommodation Tasmania.
-------------------
Retrieving Results for Index 41: aojiang.
Closest hotel in aojiang is Binjiang Hotel.
-------------------
Retrieving Results for Index 75: sao filipe.
Closest hotel in sao filipe is Tortuga B&B.
-------------------
Retrieving Results for Index 97: saint-philippe.
Closest hotel in saint-philippe is Chambres d'hôte "La Trinité".
-------------------
Retrieving Results for Index 121: ambovombe.
Closest hotel in ambovombe is MONJA Talilisoa.
-------------------
Retrieving Results for Index 215: sitrah.
Closest hotel in sitrah is Al Bander Resort - منتجع البندر.
-------------------
Retrieving Results for Index 228: irece.
Closest hotel in irece is RADAR INN HOTEL.
-------------------
Retrieving Results for Index 243: hambantota.
Closest hotel in hambantota is Bungalow 63.
-------------------
Retrieving Results for Index 257: new norfolk.
Closest hotel in new norfolk 

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
11,hobart,AU,-42.88,147.33,St Ives Hobart Accommodation Tasmania
41,aojiang,CN,22.94,116.04,Binjiang Hotel
75,sao filipe,CV,14.90,-24.50,Tortuga B&B
97,saint-philippe,RE,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
121,ambovombe,MG,-25.17,46.08,MONJA Talilisoa
215,sitrah,BH,26.15,50.62,Al Bander Resort - منتجع البندر
228,irece,BR,-11.30,-41.86,RADAR INN HOTEL
243,hambantota,LK,6.12,81.12,Bungalow 63
257,new norfolk,AU,-42.78,147.06,The Shingles Riverside Cottages
282,wonthaggi,AU,-38.60,145.59,Wonthaggi Caravans & Trailers


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))